# Take the frame based detections and turn them into movement trajectories
(For speed, uses multiprocessing and breaks up observations into 10 overlapping temporal groups to process seperately since tracks are much shorter in time than the total length of the obervations.)

In [ ]:
import glob
import os

import matplotlib.pyplot as plt
from multiprocessing import Pool
import numpy as np

import bat_functions as kbf

In [ ]:
base_folder = ".../kasanka-bats/processed/deep-learning"
# Which day to track
day = "16Nov"
camera_folders = sorted(glob.glob(os.path.join(base_folder, day, '*')))

In [ ]:
n_camera_folders = []
for folder in camera_folders:
    tracks_file = os.path.join(folder, "raw_tracks.npy")
    if os.path.exists(tracks_file):
        # Skip videos that have already been tracked
        continue
    else:
        n_camera_folders.append(folder)

print("Videos to track...")
print(*n_camera_folders, sep='\n')

In [ ]:
def track(camera_dict):
    camera_folder = camera_dict['camera_folder']
    first_frame = camera_dict['first_frame']
    max_frame = camera_dict['max_frame']
    print(f"{os.path.basename(camera_folder)} begun.")
    contours_files = sorted(
        glob.glob(os.path.join(camera_folder, 'contours-compressed-*.npy'))
    )
    if contours_files:
        contours_files = contours_files[1:]
        centers = np.load(os.path.join(camera_folder, 'centers.npy'), allow_pickle=True)
        sizes = np.load(os.path.join(camera_folder, 'size.npy'), allow_pickle=True)
        tracks_file = os.path.join(camera_folder, f'first_frame_{first_frame}_max_val_{max_frame}_raw_tracks.npy')
        raw_tracks = kbf.find_tracks(first_frame, centers, contours_files=contours_files, 
                                     sizes_list=sizes, tracks_file=tracks_file,
                                     max_frame=max_frame)
    else:
        print("Missing contour files.")

In [ ]:
camera_dicts = []
for camera_folder in n_camera_folders:
    # To speed up processing, detections found in each observation are split
    # into 10 groups by time with 15 seconds of overlap in each group
    centers_file = os.path.join(camera_folder, 'centers.npy')
    centers = np.load(centers_file, allow_pickle=True)
    max_vals = np.linspace(0, len(centers), 10, dtype=int)[1:].tolist()
    max_vals[-1] = None
    min_vals = np.linspace(0, len(centers), 10, dtype=int)[:-1]
    # 15 second overlap
    min_vals[1:] = min_vals[1:] - 450
    for min_val, max_val in zip(min_vals, max_vals):
        min_val = np.max([min_val, 0])
        camera_dict = {'camera_folder': camera_folder,
                       'first_frame': min_val,
                       'max_frame': max_val}
        if max_val is None:
            tracks_basename = f'first_frame_{min_val:06d}_max_val_{max_val}_raw_tracks.npy'
        else:
            tracks_basename = f'first_frame_{min_val:06d}_max_val_{max_val:06d}_raw_tracks.npy'
        tracks_file = os.path.join(camera_folder, tracks_basename)
        if not os.path.exists(tracks_file):
            print(tracks_file)
            camera_dicts.append(camera_dict)
        

In [ ]:
with Pool(processes=5) as pool:
    pool.map(track, camera_dicts)

### Now connect the 10 sections of the observation that were tracked seperately together

In [ ]:
def combine_overlapping_tracks(observation_folder, first_group=0, last_group=None, save=False):
    track_files = glob.glob(os.path.join(observation_folder, 'first_frame*.npy'))
    track_files = sorted(track_files, key=lambda f: int(f.split('_')[-6]))

    track_groups = []
    for file in track_files:
        track_groups.append(np.load(file, allow_pickle=True))
        
        
    for track_file in track_files:
        print(os.path.basename(track_file))
        
    first_overlap_frames = [int(f.split('_')[-6]) for f in track_files[1:]]
    first_overlap_frames.append(None)
    print(first_overlap_frames)
        
    all_tracks = []

    total_tracks = 0
    for group_ind, track_group in enumerate(track_groups[first_group:last_group]):
        if group_ind >= len(track_groups) -1:
            for track in track_group:
                if type(track['track']) == list:
                    track['track'] = np.stack(track['track'])
                    track['pos_index'] = np.stack(track['pos_index'])
                    if 'size' in track:
                        track['size'] = np.stack(track['size'])
                all_tracks.append(track)
            total_tracks += len(track_group)
            break

        for track_ind, track in enumerate(track_group):
            if track['first_frame'] < first_overlap_frames[first_group + group_ind]:
                all_tracks.append(track)


    all_tracks_file = os.path.join(observation_folder, 'raw_tracks.npy')
    if save:
        np.save(all_tracks_file, all_tracks)
        print('saved')

In [ ]:
observation_folders = []
for folder in camera_folders:
    if not os.path.exists(os.path.join(folder, 'raw_tracks.npy')):
        observation_folders.append(folder)

In [ ]:
for folder in observation_folders:
    combine_overlapping_tracks(folder, save=True)